In [1]:
# imports
import os
import json
import openai
import pickle
import getpass
from openai import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document

In [10]:
# Carrega a chave da API a partir do arquivo JSON
with open('config.json', 'r') as file:
    config = json.load(file)
    os.environ['OPENAI_API_KEY'] = config.get('OPENAI_API_KEY')

In [3]:
# Define o caminho relativo para a pasta 'splits' dentro do projeto
current_directory = os.getcwd()
splits_directory = os.path.join(current_directory, 'splits')
splits_filepath = os.path.join(splits_directory, 'splits.pkl')

In [4]:
# Função para carregar os splits
def load_splits(filepath):
    with open(filepath, 'rb') as file:
        splits = pickle.load(file)
    return splits

# Carregar os splits usando o caminho relativo
splits = load_splits(splits_filepath)

In [5]:
# Definindo os embeddings antes de carregar o FAISS
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

c:\Users\gust_\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [6]:
# Carregando o índice FAISS localmente
faiss_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [7]:
# Criar o retriever a partir do faiss_db
retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [8]:
client = OpenAI()

def generate_response(query):
    # Recuperar as 5 combinações mais relevantes
    relevant_docs = retriever.invoke(query)
    context = " ".join([doc.page_content for doc in relevant_docs])

    prompt = f"""
    Você é um assistente de perguntas e respostas. Seu objetivo é responder perguntas com a maior precisão possível com base nos documentos fornecidos. Responda sempre em português.

    Contexto: {context}

    Pergunta: {query}
    Resposta:
    """
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Você é um assistente de perguntas e respostas. Seu objetivo é responder perguntas com a maior precisão possível com base nas instruções e no contexto fornecido. Responda sempre em português."},
            {"role": "user", "content": f"Contexto: {context} Pergunta: {query}"}
        ],
        max_tokens=150,
        temperature=0.5
    )

    #print(completion.choices[0].message)
    return completion.choices[0].message.content


In [9]:
# Exemplo de uso
query = "Quem é o representante dos Recursos HUmanos do Shopping Cidadão?"
resposta = generate_response(query)
print(resposta)

Os representantes dos Recursos Humanos do Shopping Cidadão são Carlos Fernandez Rodriguez, Priscila Fonseca e Luciano Catarino. Suas informações de contato são:

- Carlos Fernandez Rodriguez | Recursos Humanos: carlos.fernandez@shopcidadao.com.br
- Priscila Fonseca | Recursos Humanos: priscila.fonseca@shopcidadao.com.br
- Luciano Catarino | Recursos Humanos: luciano.catarino@shopcidadao-mg.com.br
